# Description.ipynb
## Generates Description from Fetched Data
### LSTM Code by https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

# Imports

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM

# Global Variables

In [2]:
FILE_LOCATION = "https://raw.githubusercontent.com/alexandre-lavoie/youtube-bot/master/data/database/CA_viewCount.csv"
MIN_TITLE_LENGTH = 31
NUMBER_OF_DESCRIPTIONS = 10
DESCRIPTION_LENGTH = 500
EPOCHS = 20
BATCH_SIZE = 128

# Read and Cleanup Database

In [3]:
video_database = pd.read_csv(FILE_LOCATION, error_bad_lines=False)

In [4]:
cleanup_database = video_database.replace('[^\x00-\x7F]+','',regex=True)
cleanup_database = cleanup_database.replace('(http|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', 'URL', regex=True)
cleanup_database = cleanup_database.replace(' \.\.\.', '', regex=True)
description_database = [description for description in cleanup_database["description"] if isinstance(description,str)]

# Get Character Dictionary

In [5]:
chars = [chr(i) for i in range(ord('!'), ord('Z')+1)]
chars.extend(['|', ' ', '[', ']', '_', '~'])
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_vocabs = len(chars)

# Converts Titles Into Training Data

In [6]:
description_int = []
for description in description_database:
    description_int.append ([char_to_int[letter] for letter in description.upper()])

In [7]:
dataX = []
dataY = []

for description in description_int:
    for i in range(0, len(description)-MIN_TITLE_LENGTH):
        dataX.append(description[i:(i+MIN_TITLE_LENGTH)])
        dataY.append(description[i+MIN_TITLE_LENGTH])
        
n_patterns = len(dataX)

In [8]:
print("Vocab Length: " + str(n_vocabs))
print("Pattern Length: " + str(MIN_TITLE_LENGTH))
print("Number of patterns: " + str(n_patterns))

Vocab Length: 64
Pattern Length: 31
Number of patterns: 42061


In [9]:
X = np.reshape(dataX, (n_patterns, MIN_TITLE_LENGTH, 1))
X = X/float(n_vocabs)
y = np.eye(len(chars))[dataY]

# LSTM

In [10]:
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training

In [11]:
model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/20
42061/42061 [==============================] - 13s 312us/step - loss: 3.1347
Epoch 2/20
42061/42061 [==============================] - 10s 228us/step - loss: 3.0716
Epoch 3/20
42061/42061 [==============================] - 9s 225us/step - loss: 3.0221
Epoch 4/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.9634
Epoch 5/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.9005
Epoch 6/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.8539
Epoch 7/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.8130
Epoch 8/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.7716
Epoch 9/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.7364
Epoch 10/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.6935
Epoch 11/20
42061/42061 [==============================] - 9s 225us/step - loss: 2.6515
Epoch 12/20
42061/42061 [==============

# Predictions

In [12]:
for _ in range(NUMBER_OF_DESCRIPTIONS):
    start = np.random.randint(0, len(dataX)-1)
    patt = dataX[start]
    text = ""
    for value in patt:
        text += int_to_char[value]

    for i in range(DESCRIPTION_LENGTH):
        xx = np.reshape(patt, (1, len(patt), 1))
        xx = xx / float(n_vocabs)
        prediction = model.predict(xx, verbose=0)
        index = np.argmax(prediction)
        text += int_to_char[index]
        patt.append(index)
        patt = patt[1:len(patt)]

    print(text)

O BY THE CRANBERRIES PERFORMING THE SEAR THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER A
 DREAM OF BECOMING A PHYSICIST AND SHE ARE IN THE WORLD THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE SECO DOOMENT AND HAT AND MER ALD MER AND MN THE SECO THE 

In [14]:
model_json = model.to_json()
with open("description_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("description_model.h5")